In [1]:
import cv2 as cv
import time
import win32gui, win32ui, win32con, win32api
import numpy as np
import dxcam
from ultralytics import YOLO
from IPython.display import display, clear_output
from win32api import GetSystemMetrics
import pydirectinput
import scipy
import ctypes
import math
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction



In [2]:
#BASIC CONFIG#

FPS=True # if True - FPS and averageFPS will be displayed
VERBOSE=False #if True - detected boxes will be drawn(it will decrease FPS by a lot)


#choose one of
PREDICT=True #if True - will detect large and medium-sized targets (this is enough for most games, so it is not recommended to change this parameter)
SAHI=False #if True - will detect small and medium sized targets (this will reduce FPS by about 10x, so its an option only for rich guys with powerful GPU)

SAHI_CT=0.3 #


YOLO_CONFIDENCE_THRESHOLD=0.5 #
IOU=0.5 #

dw = win32api.GetSystemMetrics(0) #display width in pixels
dh = win32api.GetSystemMetrics(1) #display height in pixels


AIM_ASSIST=True #
TRIGGER=True #

#CHOOSE ONE OF:
ACTIVE_WHEN_KEY_PRESSED='' #
ALWAYS_ACTIVE=False #


#AIM ASSIST CONFIG(needed only if AIM_ASSIST=True)
LEFT=int(dw*0.4428) #
TOP=int(dh*0.352) #
RIGHT=int(dw*0.6095) #
BOTTOM=int(dh*0.6483) #

#If this variable is too small, the movement will be slow, if it is too high, the movement will be jerky.
#You have to determine this variable yourself by trial and error.
SCALE=0.3 #determines how big the steps will be when the aim assist brings the cursor to the target.

#Where to aim?
HEAD=False
BODY=False
RABDOM=False



In [3]:
LEFT,TOP,RIGHT,BOTTOM

(850, 380, 1170, 700)

In [4]:
dc = win32gui.GetDC(0)
dcObj = win32ui.CreateDCFromHandle(dc)
hwnd = win32gui.WindowFromPoint((0,0))
monitor = (0, 0, dw, dh)

b1 = win32ui.CreateBrush()
b1.CreateSolidBrush(win32api.GetSysColor(0))



def box_in_center(box):
    if box[0]+LEFT<dw/2 and box[2]+LEFT>dw/2 and box[1]+TOP<dh/2 and box[3]+TOP>dh/2:
        return True
    
    return False
    

def find_closest(r):
    min_distance=1000000
    closest=[]
    
    if PREDICT or TRACK:
        for box in r.boxes.xywh:
            dist=math.dist([dw/2,dh/2],[ box[0]+LEFT,box[1]+TOP] )
            if dist<min_distance:
                min_distance=dist
                closest=box[0]+LEFT,box[1]+TOP-box[3]*0.35
                
        return closest
        
    elif SAHI:
        for res in r.to_coco_annotations():
            
            if res['score']>SAHI_CT:
                x=res['bbox'][0] + res['bbox'][2]/2
                y=res['bbox'][1] + res['bbox'][3]/2
                dist=math.dist([dw/2,dh/2],[ x+LEFT,y+TOP])
                if dist<min_distance:
                    min_distance=dist
                    closest=x+LEFT,y+TOP-res['bbox'][3]*0.35
            else: 
                continue 
                
        return closest


def draw_box(left,top,right,bottom):
    dcObj.FrameRect((LEFT, TOP, RIGHT, BOTTOM), b1)
    dcObj.FrameRect((left+LEFT,top+TOP,right+LEFT,bottom+TOP), b1)
    win32gui.InvalidateRect(hwnd, monitor, True)

def move(closest):
    x = closest[0] - dw/2
    y = closest[1] - dh/2 

    scale = SCALE
    x = int(x * scale)
    y = int(y * scale)
    win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, x, y, 0, 0)

    return

def click(duratiom=0.0):
    pydirectinput.click(duration=duratiom,_pause=False)
    return

def process_yolo_result(result):
    for r in result:
        if AIM_ASSIST:
            closest=find_closest(r)
            if len(closest)>0 and  win32api.GetKeyState(0x02)<0:
                move(closest)
                        
        if TRIGGER: 
            for box in r.boxes.xyxy:
                if len(box) and  win32api.GetKeyState(0x02)<0:
                    if box_in_center(box):
                        click()                       
        
        if VERBOSE:
            for box in r.boxes.xyxy:
                box=[int(x) for x in box]
                draw_box(*box)
                
            display('Number of persons: '+str(len(r.boxes.xyxy)) )
            display('bboxes in xyxy format: '+str(r.boxes.xyxy) )

def process_sahi_result(result):
    if AIM_ASSIST:
        closest=find_closest(result)
        if len(closest)>0 and  win32api.GetKeyState(0x02)<0:
            move(closest)
        
    if TRIGGER:
        for res in result.to_coco_annotations():
            if len(res) and  win32api.GetKeyState(0x02)<0 and res['score']>SAHI_CT:
                box=[res['bbox'][0],res['bbox'][1],res['bbox'][0]+res['bbox'][2],res['bbox'][1]+res['bbox'][3]]
                if box_in_center(box):
                        click()

    if VERBOSE:
         for res in result.to_coco_annotations():
            box=[res['bbox'][0],res['bbox'][1],res['bbox'][0]+res['bbox'][2],res['bbox'][1]+res['bbox'][3]]
            box=[int(x) for x in box]
            draw_box(*box)

In [5]:
camera = dxcam.create(output_color="BGR")
camera.start(region=(LEFT, TOP, RIGHT, BOTTOM))

yolo_model = YOLO("best.pt") 

sahi_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path="best.pt",
    confidence_threshold=YOLO_CONFIDENCE_THRESHOLD,
    device="cuda:0", # or 'cuda:0'
    image_size=640,
    
)


frames_counter=0
all_time=0

In [ ]:
while(True):
    clear_output(wait=True)
    start= time.time()
    
    screenshot = camera.get_latest_frame()

    if SAHI:
        result= get_sliced_prediction(
            screenshot,
            sahi_model,
            slice_height = 128,
            slice_width = 128,
            overlap_height_ratio = 0.2,
            overlap_width_ratio = 0.2,
            verbose=True
        )

        process_sahi_result(result)
        

    if PREDICT:
        result=yolo_model.predict(screenshot,imgsz=320, conf=YOLO_CONFIDENCE_THRESHOLD,verbose=False,iou=IOU)
        process_yolo_result(result)            
    
    # cv.imshow('Computer Vision', screenshot)
    
    if FPS: 
        display('FPS: '+str(1/(time.time()-start+0.0015)) )
        frames_counter+=1
        all_time+=1/(time.time()-start)
        display('Avg FPS: '+str(all_time/frames_counter)) 
        

    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        camera.stop()
        break

'FPS: 60.11403697757177'

In [ ]:
# win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, 100, 200, 0, 0)